# 1. Separate into different sections when code is ready

In [1]:
import numpy as np
import os

In [2]:
class DataManipulation:

    # also make sure where this is called from
    MEDIAPIPE_DATA_PATH = os.path.join('../CodeForMachineWithCamera/DataOnlyHands3') # TODO: make sure to name the folder 'Data'
    SIGN_LABELS = np.array(['hello', 'yes', 'no'])
    NUMBER_OF_SEQUENCES = 50
    SEQUENCE_LENGTH = 30

    # TODO: test once code is ready
    # label_map = {label:num for num, label in enumerate(SIGN_LABELS)}

    @staticmethod
    def create_data_set():
        sequences, labels = [], []
        for sign in DataManipulation.SIGN_LABELS:
            for sequence in range(DataManipulation.NUMBER_OF_SEQUENCES):
                frame = []
                for frame_number in range(DataManipulation.SEQUENCE_LENGTH):
                    frame.append(
                        np.load(os.path.join(DataManipulation.MEDIAPIPE_DATA_PATH,
                                             sign,
                                             str(sequence),
                                             "{}.npy".format(frame_number))))
                sequences.append(frame)
                labels.append(np.where(DataManipulation.SIGN_LABELS == sign)[0][0])
                # TODO: test once code is ready
                # LABELS.append(label_map[sign])

        X = np.array(sequences)
        y = to_categorical(labels).astype(int)

        return train_test_split(X, y, test_size=0.05) # may wanna play with this size

In [3]:
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard

2024-03-18 16:57:25.647279: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-18 16:57:25.690421: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-18 16:57:26.304283: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
class Model:
    # put this in the same directory with the model dir
    LOG_DIR = os.path.join('Logs')
    CALLBACK = TensorBoard(log_dir=LOG_DIR) # used to save information of the training process

    def build_model():
        # may wanna play with the parameters of the following lines of code
        model = Sequential()
        model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 126))) # 126 only hands, 1662 with face and pose
        model.add(LSTM(128, return_sequences=True, activation='relu'))
        model.add(LSTM(64, return_sequences=False, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(DataManipulation.SIGN_LABELS.shape[0], activation='softmax'))

        return model

In [5]:
X_train, X_test, y_train, y_test = DataManipulation.create_data_set()

model = Model.build_model()
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(X_train, y_train, epochs=2000, callbacks=[Model.CALLBACK])
model.summary()

2024-03-18 16:57:27.634076: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46851 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:3b:00.0, compute capability: 8.6
2024-03-18 16:57:27.634749: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46864 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:af:00.0, compute capability: 8.6


Epoch 1/2000


2024-03-18 16:57:31.101726: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-03-18 16:57:31.233662: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7a9da0001a10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-18 16:57:31.233707: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2024-03-18 16:57:31.233722: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (1): NVIDIA RTX A6000, Compute Capability 8.6
2024-03-18 16:57:31.245515: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-18 16:57:31.342916: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8907
2024-03-18 16:57:31.45859

5/5 [==============================] - 5s 161ms/step - loss: 1.0973 - categorical_accuracy: 0.4014
Epoch 2/2000
5/5 [==============================] - 1s 157ms/step - loss: 1.0786 - categorical_accuracy: 0.4366
Epoch 3/2000
5/5 [==============================] - 1s 160ms/step - loss: 1.0323 - categorical_accuracy: 0.4225
Epoch 4/2000
5/5 [==============================] - 1s 156ms/step - loss: 0.9570 - categorical_accuracy: 0.5634
Epoch 5/2000
5/5 [==============================] - 1s 149ms/step - loss: 0.8675 - categorical_accuracy: 0.6690
Epoch 6/2000
5/5 [==============================] - 1s 164ms/step - loss: 0.8474 - categorical_accuracy: 0.6268
Epoch 7/2000
5/5 [==============================] - 1s 169ms/step - loss: 0.6732 - categorical_accuracy: 0.7746
Epoch 8/2000
5/5 [==============================] - 1s 162ms/step - loss: 0.6608 - categorical_accuracy: 0.7676
Epoch 9/2000
5/5 [==============================] - 1s 166ms/step - loss: 0.6948 - categorical_accuracy: 0.7746
Epoch

In [6]:
model.save('Model8.h5')

/home/aicore/miniconda3/envs/actionRec/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
model.save('Model8.keras')